In [83]:
import gsd.hoomd
import numpy as np

In [84]:
def compute_all_flake_coms(flake_positions, box_lengths, beads_per_flake=50):
    Lx, Ly, Lz = box_lengths
    n_flakes = len(flake_positions) // beads_per_flake
    flake_coms = []

    for i in range(n_flakes):
        flake = flake_positions[i * beads_per_flake : (i + 1) * beads_per_flake]
        ref = flake[0]
        delta = flake - ref

        # Minimum image convention
        delta[:, 0] -= Lx * np.round(delta[:, 0] / Lx)
        delta[:, 1] -= Ly * np.round(delta[:, 1] / Ly)
        delta[:, 2] -= Lz * np.round(delta[:, 2] / Lz)

        unwrapped = ref + delta
        com = np.mean(unwrapped, axis=0)

        # Wrap CoM into [-L/2, L/2]
        com[0] = (com[0] + Lx/2) % Lx - Lx/2
        com[1] = (com[1] + Ly/2) % Ly - Ly/2
        com[2] = (com[2] + Lz/2) % Lz - Lz/2

        flake_coms.append(com)

    return np.array(flake_coms)


In [85]:
def compute_all_flake_coms_from_38th(flake_positions, box_lengths, beads_per_flake=50):
    Lx, Ly, Lz = box_lengths
    n_flakes = len(flake_positions) // beads_per_flake
    flake_coms = []

    for i in range(n_flakes):
        idx = i * beads_per_flake + 37
        pos = np.array(flake_positions[idx])

        # Wrap position into [-L/2, L/2]
        pos[0] = (pos[0] + Lx / 2) % Lx - Lx / 2
        pos[1] = (pos[1] + Ly / 2) % Ly - Ly / 2
        pos[2] = (pos[2] + Lz / 2) % Lz - Lz / 2

        flake_coms.append(pos)

    return np.array(flake_coms)

In [86]:
def find_flake_neighbors(coms, box_lengths, cutoff):
    Lx, Ly, Lz = box_lengths
    n = len(coms)
    for i in range(n):
        neighbors = []
        for j in range(n):
            if i == j:
                continue
            dr = coms[j] - coms[i]
            dr[0] -= Lx * np.round(dr[0] / Lx)
            dr[1] -= Ly * np.round(dr[1] / Ly)
            dr[2] -= Lz * np.round(dr[2] / Lz)
            dist = np.linalg.norm(dr)
            if dist < cutoff:
                neighbors.append((j, dist))
        if neighbors:
            neighbor_str = ", ".join([f"Flake {j} (distance: {d:.2f})" for j, d in neighbors])
            print(f"Flake {i} is neighbors with: {neighbor_str}")
        else:
            print(f"Flake {i} has no neighbors within cutoff range.")

In [87]:
traj = gsd.hoomd.open("100_10mer10f_0.0005dt_7kT_large.gsd")
frame = traj[93220]  # frame index

In [92]:
flake_type = "F"
flake_typeid = frame.particles.types.index(flake_type)
positions = frame.particles.position
typeids = frame.particles.typeid
flake_positions = positions[typeids == flake_typeid]

Lx, Ly, Lz = frame.configuration.box[:3]

flake_coms = compute_all_flake_coms_from_38th(flake_positions, (Lx, Ly, Lz), beads_per_flake=50)
find_flake_neighbors(flake_coms, (Lx, Ly, Lz), cutoff=2.5)

Flake 0 has no neighbors within cutoff range.
Flake 1 has no neighbors within cutoff range.
Flake 2 has no neighbors within cutoff range.
Flake 3 has no neighbors within cutoff range.
Flake 4 has no neighbors within cutoff range.
Flake 5 has no neighbors within cutoff range.
Flake 6 is neighbors with: Flake 8 (distance: 1.11), Flake 9 (distance: 2.44)
Flake 7 has no neighbors within cutoff range.
Flake 8 is neighbors with: Flake 6 (distance: 1.11), Flake 9 (distance: 1.34)
Flake 9 is neighbors with: Flake 6 (distance: 2.44), Flake 8 (distance: 1.34)


In [93]:
print(Lx,Ly,Lz)

17.09976 17.09976 17.09976


In [94]:
print(flake_coms)

[[-1.8657905   7.174376    0.13797691]
 [-6.596167    7.675875   -7.200569  ]
 [ 8.170121   -1.5282594   3.7152588 ]
 [ 4.464428    2.351693    5.5135465 ]
 [-1.7335142  -5.094418    1.9472373 ]
 [-2.6565433  -4.2195535  -3.1058908 ]
 [ 4.866673   -2.4554129   2.6737108 ]
 [-4.514638    2.8086731  -7.7552233 ]
 [ 4.31046    -1.4952607   2.7449508 ]
 [ 3.8405015  -0.24392131  2.6731768 ]]


In [38]:
flake_coms[7]

array([0.58595276, 7.2629943 , 8.231641  ], dtype=float32)

In [39]:
len(flake_coms)

10

In [45]:
flake_coms[9]

array([-0.8008323, -4.3493123, -6.192117 ], dtype=float32)

In [47]:
flake_coms[7]

array([0.58595276, 7.2629943 , 8.231641  ], dtype=float32)

In [50]:
def mic_distance(pos1, pos2, box_lengths):
    Lx, Ly, Lz = box_lengths
    dr = pos1 - pos2
    dr[0] -= Lx * np.round(dr[0] / Lx)
    dr[1] -= Ly * np.round(dr[1] / Ly)
    dr[2] -= Lz * np.round(dr[2] / Lz)
    return np.linalg.norm(dr)

In [65]:
box_lengths = (17.09976, 17.09976, 17.09976)
distance = mic_distance(flake_coms[8], flake_coms[9], box_lengths)
print(distance)

2.165086


In [66]:
distance = np.linalg.norm(flake_coms[8] - flake_coms[9])
print(distance)

2.165086


In [67]:
print(flake_coms[8])

[ 4.5307055 -1.2893027  4.8112273]


In [68]:
print(flake_coms[9])

[ 4.0351806  -0.09554996  6.5481806 ]
